In [1]:
import PISM
import os
import numpy as np
from PISM.testing import *

import numpy as np
import pylab as plt

config = PISM.Context().config

# reduce the grid size to speed this up
config.set_number("grid.Mx", 3)
config.set_number("grid.My", 5)
config.set_number("grid.Mz", 5)

seconds_per_year = 365 * 86400
# ensure that this is the correct year length
config.set_string("time.calendar", "365_day")
config.set_flag("ocean.three_equation_model_clip_salinity", False)

# silence models' initialization messages
PISM.Context().log.set_threshold(1)

ice_density = config.get_number("constants.ice.density")
sea_water_density = config.get_number("constants.sea_water.density")

In [2]:
def melting_point_temperature(ice_thickness, salinity):
        a = [-0.0575, 0.0901, -7.61e-4]
        return a[0] * salinity + a[1] + a[2] * ice_thickness

In [3]:
h = np.arange(800, 1000, 100)
t = np.arange(4, 6, 1)
H, T = np.meshgrid(h, t)
nx, ny = T.shape

M = np.zeros_like(H)

gamma_T = 1.00e-4
gamma_S = 5.05e-7


for ice_thickness in h:
    for temperature in t:
        depth = ice_density / sea_water_density * ice_thickness
        salinity = 35.0
        melange_back_pressure = 0.0
        potential_temperature = temperature - melting_point_temperature(ice_thickness, salinity) + 273.13

        grid = shallow_grid()
        geometry = PISM.Geometry(grid)

        geometry.ice_thickness.set(depth)

        filename = "ocean_given_th_input.nc"

        PISM.util.prepare_output(filename)

        Th = PISM.IceModelVec2S(grid, "theta_ocean", PISM.WITHOUT_GHOSTS)
        Th.set_attrs("climate", "potential temperature", "Kelvin", "Kelvin", "", 0)
        Th.set(potential_temperature)
        Th.write(filename)

        S = PISM.IceModelVec2S(grid, "salinity_ocean", PISM.WITHOUT_GHOSTS)
        S.set_attrs("climate", "ocean salinity", "g/kg", "g/kg", "", 0)
        S.set(salinity)
        S.write(filename)

        config.set_string("ocean.th.file", filename)
        config.set_number("ocean.th.gamma_T", gamma_T)
        config.set_number("ocean.th.gamma_S", gamma_S)

        model = PISM.OceanGivenTH(grid)
        model.init(geometry)
        model.update(geometry, 0, 1)
        m = model.shelf_base_mass_flux().numpy()[0, 0]
        m = m * seconds_per_year / ice_density
        print(f"H={ice_thickness} m, TH={potential_temperature-273.13:.2f} C, S={salinity} g/kg, m={m:.2f} m/yr")


H=800 m, TH=6.53 C, S=35.0 g/kg, m=139.43 m/yr
H=800 m, TH=7.53 C, S=35.0 g/kg, m=139.43 m/yr
H=900 m, TH=6.61 C, S=35.0 g/kg, m=139.43 m/yr
H=900 m, TH=7.61 C, S=35.0 g/kg, m=139.43 m/yr


In [4]:
        gamma_T = 1.00e-4
        gamma_S = 5.05e-7

        ice_thickness = 200
        depth = ice_density / sea_water_density * ice_thickness
        salinity = 34.0
        melange_back_pressure = 0.0
        potential_temperature = 6.5 + 273.13
        temperature = 3
        potential_temperature = temperature - melting_point_temperature(ice_thickness, salinity) + 273.13

        grid = shallow_grid()
        geometry = PISM.Geometry(grid)

        geometry.ice_thickness.set(depth)

        filename = "ocean_given_th_input.nc"

        PISM.util.prepare_output(filename)

        Th = PISM.IceModelVec2S(grid, "theta_ocean", PISM.WITHOUT_GHOSTS)
        Th.set_attrs("climate", "potential temperature", "Kelvin", "Kelvin", "", 0)
        Th.set(potential_temperature)
        Th.write(filename)

        S = PISM.IceModelVec2S(grid, "salinity_ocean", PISM.WITHOUT_GHOSTS)
        S.set_attrs("climate", "ocean salinity", "g/kg", "g/kg", "", 0)
        S.set(salinity)
        S.write(filename)

        config.set_string("ocean.th.file", filename)
        config.set_number("ocean.th.gamma_T", gamma_T)
        config.set_number("ocean.th.gamma_S", gamma_S)


        model = PISM.OceanGivenTH(grid)
        model.init(geometry)
        model.update(geometry, 0, 1)
        m = model.shelf_base_mass_flux().numpy()[0, 0]
        m = m * seconds_per_year / ice_density
        print(f"H={ice_thickness} m, TH={potential_temperature-273.13:.2f} C, S={salinity} g/kg, m={m:.2f} m/yr")



H=200 m, TH=5.02 C, S=34.0 g/kg, m=134.93 m/yr


In [5]:
35900 / 362.5

99.03448275862068